In [1]:
!pip install tensorflow==1.0

In [0]:
# Importing the Libraries

import numpy as np
import tensorflow as tf
import re
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import pandas as pd
dataset = pd.DataFrame(columns=['dialogueID', 'text'])
dataset_index = 0
full_dataset = pd.read_csv("drive/My Drive/BE Project/dialogueText_196.csv")
full_dataset.columns = ['folder', 'dialogueID', 'date', 'fr_om', 'to', 'text']
full_dataset = full_dataset.drop(['folder', 'date', 'fr_om', 'to'], axis = 1)
for index, row in full_dataset.iterrows():
  if str(row['text']).lower().count("java") > 0:
    dataset.loc[dataset_index] = [row['dialogueID'], row['text']]
    dataset_index += 1
    if dataset_index == 5000:
      break
dataset.head(5)

,dialogueID,text
0,1.tsv,any ideas why java plugin takes so long to load?
1,1.tsv,java 1.4?
2,1.tsv,java 1.5 loads _much_ faster
3,1.tsv,"thanks, i'll try that, screwed up and tried to..."
4,1.tsv,1.42 is latest IBM java?


In [0]:
# Adding an id column

dataset_size = dataset.shape[0]
id_column = []
for n in range(0, dataset_size):
  id_column.append("L" + str(n))

dataset['id'] = id_column

In [7]:
print(dataset_size)
dataset.head(5)

5000


,dialogueID,text,id
0,1.tsv,any ideas why java plugin takes so long to load?,L0
1,1.tsv,java 1.4?,L1
2,1.tsv,java 1.5 loads _much_ faster,L2
3,1.tsv,"thanks, i'll try that, screwed up and tried to...",L3
4,1.tsv,1.42 is latest IBM java?,L4


In [0]:
# Dictionary that maps each line and its id

id2line = {}
for index, row in dataset.iterrows():
  id2line[row['id']] = row['text']

In [9]:
print(id2line['L0'])
print(id2line['L3'])

any ideas why java plugin takes so long to load?
thanks, i'll try that, screwed up and tried to upgrade firefox/java on my own now java takes real long time to load


In [0]:
# List of all conversations

conversations_ids = []
inside_list = []
o_d_id = ""
for index, row in dataset.iterrows():
  if index == 0:
    inside_list.append(row['id'])
    o_d_id = str(row['dialogueID'])
    continue
  i_d_id = str(row['dialogueID'])
  if i_d_id == o_d_id:
    inside_list.append(row['id'])
  else:
    o_d_id = i_d_id
    conversations_ids.append(inside_list)
    inside_list = []
    inside_list.append(row['id'])
conversations_ids.append(inside_list)

In [11]:
print(conversations_ids[2:5])

[['L9', 'L10', 'L11', 'L12', 'L13'], ['L14', 'L15', 'L16', 'L17', 'L18', 'L19', 'L20', 'L21', 'L22', 'L23', 'L24', 'L25', 'L26', 'L27', 'L28', 'L29'], ['L30', 'L31']]


In [0]:
# Getting questions and answers separately

questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i + 1]])

In [13]:
print(questions[:5])
print(answers[:5])

['any ideas why java plugin takes so long to load?', 'java 1.4?', 'java 1.5 loads _much_ faster', "thanks, i'll try that, screwed up and tried to upgrade firefox/java on my own now java takes real long time to load", '1.42 is latest IBM java?']
['java 1.4?', 'java 1.5 loads _much_ faster', "thanks, i'll try that, screwed up and tried to upgrade firefox/java on my own now java takes real long time to load", '1.42 is latest IBM java?', "do you have j2se-common installed in ubuntu as well is ibm's java?"]


In [0]:
# First cleaning of the text

def clean_text(text):
  text = text.lower()
  text = re.sub(r"aren't", "are not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"couldn't", "could not", text)
  text = re.sub(r"didn't", "did not", text)
  text = re.sub(r"doesn't", "does not", text)
  text = re.sub(r"don't", "do not", text)
  text = re.sub(r"hadn't", "had not", text)
  text = re.sub(r"hasn't", "has not", text)
  text = re.sub(r"haven't", "have not", text)
  text = re.sub(r"he'd", "he had", text)
  text = re.sub(r"he'll", "he will", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"I'd", "I had", text)
  text = re.sub(r"I'll", "I will", text)
  text = re.sub(r"I'm", "I am", text)
  text = re.sub(r"I've", "I have", text)
  text = re.sub(r"isn't", "is not", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"let's", "let us", text)
  text = re.sub(r"mustn't", "must not", text)
  text = re.sub(r"shan't", "shall not", text)
  text = re.sub(r"she'd", "she had", text)
  text = re.sub(r"she'll", "she will", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"shouldn't", "should not", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"there's", "there is", text)
  text = re.sub(r"they'd", "they had", text)
  text = re.sub(r"they'll", "they will", text)
  text = re.sub(r"they're", "they are", text)
  text = re.sub(r"they've", "they have", text)
  text = re.sub(r"we'd", "we had", text)
  text = re.sub(r"we're", "we are", text)
  text = re.sub(r"we've", "we have", text)
  text = re.sub(r"weren't", "were not", text)
  text = re.sub(r"what'll", "what will", text)
  text = re.sub(r"what're", "what are", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"what've", "what have", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"who'll", "who will", text)
  text = re.sub(r"who're", "who are", text)
  text = re.sub(r"who's", "who is", text)
  text = re.sub(r"who've", "who have", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"wouldn't", "would not", text)
  text = re.sub(r"you'd", "you had", text)
  text = re.sub(r"you'll", "you will", text)
  text = re.sub(r"you're", "you are", text)
  text = re.sub(r"you've", "you have", text)
  text = re.sub(r"&", "and", text)
  text = re.sub(r"[~`!@#$%^*()-_+={}:;'<,>.?/\|]", "", text)
  text = re.sub(r"  ", " ", text)
  if(text.endswith(" ") or text.startswith(" ")):
    text = re.sub(r" ", "", text)
  return text

In [0]:
# Cleaning the questions and the answers

clean_questions = []
for question in questions:
  clean_questions.append(clean_text(str(question)))

clean_answers = []
for answer in answers:
  clean_answers.append(clean_text(str(answer)))

In [16]:
print(clean_questions[:5])
print(clean_answers[:5])

['any ideas why java plugin takes so long to load', 'java', 'java loads much faster', 'thanks ill try that screwed up and tried to upgrade firefoxjava on my own now java takes real long time to load', 'islatestibmjava']
['java', 'java loads much faster', 'thanks ill try that screwed up and tried to upgrade firefoxjava on my own now java takes real long time to load', 'islatestibmjava', 'do you have jsecommon installed in ubuntu as well is ibms java']


In [0]:
# Dictionary that maps each word to its number of occurences

word2count = {}
for question in clean_questions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1
for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1

In [18]:
print(word2count)

{'any': 103, 'ideas': 13, 'why': 74, 'java': 3561, 'plugin': 199, 'takes': 10, 'so': 232, 'long': 12, 'to': 1561, 'load': 30, 'loads': 10, 'much': 36, 'faster': 6, 'thanks': 33, 'ill': 25, 'try': 199, 'that': 646, 'screwed': 4, 'up': 85, 'and': 1084, 'tried': 59, 'upgrade': 16, 'firefoxjava': 2, 'on': 352, 'my': 253, 'own': 6, 'now': 150, 'real': 16, 'time': 39, 'islatestibmjava': 2, 'do': 588, 'you': 1250, 'have': 622, 'jsecommon': 2, 'installed': 429, 'in': 687, 'ubuntu': 175, 'as': 195, 'well': 145, 'is': 1349, 'ibms': 5, 'i': 1972, 'not': 775, 'know': 174, 'what': 345, 'no': 201, 'matter': 15, 'use': 224, 'it': 1165, 'a': 923, 'really': 51, 'applet': 52, 'problems': 10, 'smeared': 1, 'display': 10, 'some': 91, 'pages': 5, 'with': 431, 'others': 14, 'idonotknowwhatisjvmiinstalledjavaruntime': 2, 'the': 1934, 'output': 37, 'of': 402, 'updatejavaalternatives': 47, 'l': 25, 'run': 214, 'sudo': 458, 's': 60, 'javasun': 34, 'nwhen': 1, 'connect': 13, 'vnc': 3, 'server': 30, 'via': 16, 'b

In [0]:
# Dictionaries that map questions words and answers words to a unique integer

threshold = 0
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
  if count >= threshold:
    questionswords2int[word] = word_number
    word_number += 1
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
  if count>= threshold:
    answerswords2int[word] = word_number
    word_number += 1

In [20]:
print(questionswords2int['not'])

42


In [0]:
# Adding tokens

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
  questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
  answerswords2int[token] = len(answerswords2int) + 1

In [0]:
# Inverse dictionary of answerswords2int

answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

In [0]:
# Adding <EOS> token

for i in range(len(clean_answers)):
  clean_answers[i] += ' <EOS>'

In [24]:
print(clean_answers[:5])

['java <EOS>', 'java loads much faster <EOS>', 'thanks ill try that screwed up and tried to upgrade firefoxjava on my own now java takes real long time to load <EOS>', 'islatestibmjava <EOS>', 'do you have jsecommon installed in ubuntu as well is ibms java <EOS>']


In [0]:
# Translating all the questions and answers into integers
# Replacing all the words that were filtered out by <OUT> token

questions_into_int = []
for question in clean_questions:
  ints = []
  for word in question.split():
    if word not in questionswords2int:
      ints.append(questionswords2int['<OUT>'])
    else:
      ints.append(questionswords2int[word])
  questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
  ints = []
  for word in answer.split():
    if word not in answerswords2int:
      ints.append(answerswords2int['<OUT>'])
    else:
      ints.append(answerswords2int[word])
  answers_into_int.append(ints)

In [26]:
print(questions_into_int[:2])

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [3]]


In [0]:
# Sorting questions and answers by the length of the questions

sorted_clean_questions = []
sorted_clean_asnwers = []
for length in range(1, 26):
  for i in enumerate(questions_into_int):
    if len(i[1]) == length:
      sorted_clean_questions.append(questions_into_int[i[0]])
      sorted_clean_asnwers.append(answers_into_int[i[0]])

In [28]:
print(sorted_clean_questions[:5])

[[3], [29], [59], [138], [142]]


In [0]:
# Creating placeholders for inputs and targets

def model_inputs():
  inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
  targets = tf.placeholder(tf.int32, [None, None], name = 'target')
  lr = tf.placeholder(tf.float32, name = 'learning_rate')
  keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
  return inputs, targets, lr, keep_prob

In [0]:
# Pre-processing the targets

def preprocess_targets(targets, word2int, batch_size):
  left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
  right_side = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1])
  preprocessed_targets = tf.concat([left_side, right_side], 1)
  return preprocessed_targets

In [0]:
# Creating Encoder RNN Layer

def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
  lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
  lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
  encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
  _, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                     cell_bw = encoder_cell,
                                                     sequence_length = sequence_length,
                                                     inputs = rnn_inputs,
                                                     dtype = tf.float32)
  return encoder_state

In [0]:
# Decoding the training set

def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
  attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states,
                                                                                                                                  attention_option = 'bahdanau',
                                                                                                                                  num_units = decoder_cell.output_size)
  training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                            attention_keys,
                                                                            attention_values,
                                                                            attention_score_function,
                                                                            attention_construct_function,
                                                                            name = "attn_dec_train")
  decoder_output, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                training_decoder_function,
                                                                decoder_embedded_input,
                                                                sequence_length,
                                                                scope = decoding_scope)
  decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
  return output_function(decoder_output_dropout)

In [0]:
# Decoding the test/validaiton set

def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
  attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states,
                                                                                                                                  attention_option = 'bahdanau',
                                                                                                                                  num_units = decoder_cell.output_size)
  test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                            encoder_state[0],
                                                                            attention_keys,
                                                                            attention_values,
                                                                            attention_score_function,
                                                                            attention_construct_function,
                                                                            decoder_embeddings_matrix, 
                                                                            sos_id, 
                                                                            eos_id, 
                                                                            maximum_length,
                                                                            num_words,
                                                                            name = "attn_dec_inf")
  test_predictions, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                  test_decoder_function,
                                                                  scope = decoding_scope)
  return test_predictions

In [0]:
# Creating the Decoder RNN

def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
  with tf.variable_scope("decoding") as decoding_scope:
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    weights = tf.truncated_normal_initializer(stddev = 0.1)
    biases = tf.zeros_initializer()
    output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                  num_words,
                                                                  None,
                                                                  scope = decoding_scope,
                                                                  weights_initializer = weights,
                                                                  biases_initializer = biases)
    training_predictions = decode_training_set(encoder_state,
                                               decoder_cell,
                                               decoder_embedded_input,
                                               sequence_length,
                                               decoding_scope,
                                               output_function,
                                               keep_prob,
                                               batch_size)
    decoding_scope.reuse_variables()
    test_predictions = decode_test_set(encoder_state,
                                       decoder_cell,
                                       decoder_embeddings_matrix,
                                       word2int['<SOS>'],
                                       word2int['<EOS>'],
                                       sequence_length - 1,
                                       num_words,
                                       decoding_scope,
                                       output_function,
                                       keep_prob,
                                       batch_size)
  return training_predictions, test_predictions

In [0]:
# Building the seq2seq model

def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
  encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                            answers_num_words + 1,
                                                            encoder_embedding_size,
                                                            initializer = tf.random_uniform_initializer(0, 1))
  encoder_state = encoder_rnn(encoder_embedded_input,
                              rnn_size,
                              num_layers,
                              keep_prob,
                              sequence_length)
  preprocessed_targets = preprocess_targets(targets, 
                                            questionswords2int, 
                                            batch_size)
  decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
  decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix,
                                                  preprocessed_targets)
  training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                       decoder_embeddings_matrix, 
                                                       encoder_state,
                                                       questions_num_words,
                                                       sequence_length,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int,
                                                       keep_prob,
                                                       batch_size)
  return training_predictions, test_predictions

In [0]:
# Setting the Hyperparameters

epochs = 50 # default: 100
batch_size = 64 # default: 64
rnn_size = 256 # decrease to prevent overfitting dafault: 512
num_layers = 3 # decrease to prevent overfitting dafault: 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5 # default: 0.5

In [0]:
# Defining a Session

tf.reset_default_graph()
session = tf.InteractiveSession()

In [0]:
# Loading the model inputs

inputs, targets, lr, keep_prob = model_inputs()

In [0]:
# Setting the sequence length

sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

In [0]:
# Getting the shape of input tensor

input_shape = tf.shape(inputs)

In [0]:
# Getting the training and test predictions

training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)

In [0]:
# Setting up the Loss Error, the Optimizer and Gradient Clipping

with tf.name_scope("optimization"):
  loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions, 
                                                targets,
                                                tf.ones([input_shape[0], sequence_length]))
  optimizer = tf.train.AdamOptimizer(learning_rate)
  gradients = optimizer.compute_gradients(loss_error)
  clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
  optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

In [0]:
# Padding sequences with <PAD> token

# max_sequence_length = 0
def apply_padding(batch_of_sequences, word2int):
  global max_sequence_length
  max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
  return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]

In [0]:
# Splitting data into batches of questions and answers

def split_into_batches(questions, answers, batch_size):
  for batch_index in range(0, len(questions) // batch_size):
    start_index = batch_index * batch_size
    questions_in_batch = questions[start_index : start_index + batch_size]
    answers_in_batch = answers[start_index : start_index + batch_size]
    padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
    padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
    yield padded_questions_in_batch, padded_answers_in_batch

In [0]:
# Splitting questions and answers into training and test/validation sets

training_validation_split = int(len(sorted_clean_questions) * 0.15)
training_questions = sorted_clean_questions[training_validation_split : ]
training_answers = sorted_clean_asnwers[training_validation_split : ]
validation_questions = sorted_clean_questions[ : training_validation_split]
validation_answers = sorted_clean_asnwers[ : training_validation_split]

In [0]:
# Training

batch_index_check_training_loss = 16 # default: 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 50 # default: 50 or 1000
checkpoint = "drive/My Drive/BE Project/chatbot_weights_java.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
  for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
    starting_time = time.time()
    _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch, 
                                                                                           targets: padded_answers_in_batch, 
                                                                                           lr: learning_rate, 
                                                                                           sequence_length: padded_answers_in_batch.shape[1],
                                                                                           keep_prob: keep_probability})
    total_training_loss_error += batch_training_loss_error
    ending_time = time.time()
    batch_time = ending_time - starting_time
    if batch_index % batch_index_check_training_loss == 0:
      print('Epoch: {:>3} / {}, Batch: {:>4} / {}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                     epochs,
                                                                                                                                     batch_index,
                                                                                                                                     len(training_questions) // batch_size,
                                                                                                                                     total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                     int(batch_time * batch_index_check_training_loss)))
      total_training_loss_error = 0
    if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
      total_validation_loss_error = 0
      starting_time = time.time()
      for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
        batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch, 
                                                               targets: padded_answers_in_batch, 
                                                               lr: learning_rate, 
                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                               keep_prob: 1})
        total_validation_loss_error += batch_validation_loss_error
      ending_time = time.time()
      batch_time = ending_time - starting_time
      average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
      print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
      learning_rate *= learning_rate_decay
      if learning_rate < min_learning_rate:
        learning_rate = min_learning_rate
      list_validation_loss_error.append(average_validation_loss_error)
      if average_validation_loss_error <= min(list_validation_loss_error):
        print('I speak BETTER now !!')
        early_stopping_check = 0
        saver = tf.train.Saver()
        saver.save(session, checkpoint)
      else:
        print('Sorry I do not speak better, I need to practice more .')
        early_stopping_check += 1
        if early_stopping_check == early_stopping_stop:
          break
  if early_stopping_check == early_stopping_stop:
    print("My apologies, I cannot speak better anymore. This is the best I can do .")
    break
print("GAME OVER")

Epoch:   1 / 50, Batch:    0 / 32, Training Loss Error:  0.516, Training Time on 100 Batches: 47 seconds
Validation Loss Error:  1.412, Batch Validation Time: 7 seconds
I speak BETTER now !!
Epoch:   1 / 50, Batch:   16 / 32, Training Loss Error:  3.210, Training Time on 100 Batches: 128 seconds
Validation Loss Error:  1.339, Batch Validation Time: 7 seconds
I speak BETTER now !!
Epoch:   2 / 50, Batch:    0 / 32, Training Loss Error:  1.843, Training Time on 100 Batches: 68 seconds
Validation Loss Error:  1.325, Batch Validation Time: 7 seconds
I speak BETTER now !!
Epoch:   2 / 50, Batch:   16 / 32, Training Loss Error:  1.667, Training Time on 100 Batches: 162 seconds
Validation Loss Error:  1.312, Batch Validation Time: 7 seconds
I speak BETTER now !!
Epoch:   3 / 50, Batch:    0 / 32, Training Loss Error:  1.751, Training Time on 100 Batches: 73 seconds
Validation Loss Error:  1.327, Batch Validation Time: 7 seconds
Sorry I do not speak better, I need to practice more .
Epoch:   3

In [0]:
# Loading the weights and running the session

checkpoint = "drive/My Drive/BE Project/chatbot_weights_java.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)

In [0]:
# Converting questions from strings to lists of integers

def convert_string2int(question, word2int):
  question = clean_text(question)
  return [word2int.get(word, word2int['<OUT>']) for word in question.split()]

In [0]:
# Setting up the chat


while(True):
  question = input("You: ")
  if question == "Goodbye":
    break
  question = convert_string2int(question, questionswords2int)
  global max_sequence_length
  question = question + [questionswords2int['<PAD>']] * (49 - len(question)) # CHANGE
  print("Question: {}".format(question))
  fake_batch = np.zeros((batch_size, 49))
  fake_batch[0] = question
  predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
  
  answer = ''
  for i in np.argmax(predicted_answer, 1):
    if answersints2word[i] == 'i':
      token = 'I'
    elif answersints2word[i] == '<EOS>':
      token = '.'
    else:
      token = ' ' + answersints2word[i]
    answer += token
    if token == '.':
      break
  print('Chatbot: ' + answer)